In [1]:
#import dependencies
from matplotlib import pyplot as plt
from scipy.stats import linregress
import numpy as np
from sklearn import datasets
import pandas as pd

In [2]:
#import data files
reviews_path = "Resources/reviews_LA.csv"
listings_path = "Resources/listings_LA.csv"
neighborhoods_path = "Resources/neighbourhoods_LA.csv"


In [30]:
# Read data into Pandas
review_metadata = pd.read_csv(reviews_path)
listings_metadata = pd.read_csv(listings_path)
neighborhoods_metadata = pd.read_csv(neighborhoods_path)


In [75]:
# Filter by Neighborhood group to only include City of Los Angles
listings_metadata = listings_metadata.loc[listings_metadata["neighbourhood_group"]=="City of Los Angeles",:]
listings_metadata = listings_metadata.fillna(0)
# Filter to include only AirBnBs with reviews
filtered_listings_metadata = listings_metadata.loc[listings_metadata["number_of_reviews"]!=0,:]

In [76]:
# Filter by Long Term Rental
long_term_rentals = listings_metadata.loc[listings_metadata["minimum_nights"]>=30,:]
long_term_rentals.tail()


,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license
42405,689947491478146788,Echo Park Flat with Patio Garden!,1389255,Phoebe,City of Los Angeles,Echo Park,34.08856,-118.25513,Entire home/apt,250,30,1,2022-11-17,0.27,3,341,1,0
42410,610848397189198077,East Hollywood - Melrose Living,451963672,Ari,City of Los Angeles,East Hollywood,34.08831,-118.30565,Entire home/apt,70,30,1,2022-08-31,0.16,9,315,1,0
42419,46698658,"Spacious condo, spectacular Echo Park lake views!",3715553,Tim,City of Los Angeles,Echo Park,34.07123,-118.26313,Entire home/apt,145,30,10,2022-12-01,0.38,1,75,3,0
42424,8883210,Hollywood - Single private room,13158011,Nadia & Pavel,City of Los Angeles,East Hollywood,34.09502,-118.29347,Private room,45,31,9,2021-04-01,0.14,7,312,0,0
42435,32286224,Private 1 Bedroom Home: 1 Mile to Cabrillo Beach,242337078,Amanda,City of Los Angeles,San Pedro,33.71647,-118.29050,Entire home/apt,105,30,23,2022-11-21,0.47,2,93,4,0


In [82]:
ltr_average_income = long_term_rentals["price"]*long_term_rentals["minimum_nights"]*long_term_rentals["reviews_per_month"]*12
long_term_rentals["yearly_income"] = ltr_average_income
long_term_rentals.head()

/var/folders/mr/nzdp38cj3cz3b0gcgyvws4c40000gp/T/ipykernel_81479/3903615621.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  long_term_rentals["yearly_income"] = ltr_average_income


,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license,yearly_income
64,37542116,"Private guesthouse CSUN, Studio City, Hollywood",283511617,Cynthia,City of Los Angeles,Granada Hills,34.27339,-118.49232,Entire home/apt,135,30,57,2020-08-23,1.31,1,365,0,HSR19-004213,63666.0
85,50031523,Ocean/Mountain/Getty View. Walk to beach. Rare...,403528032,Sean,City of Los Angeles,Pacific Palisades,34.04404,-118.56274,Entire home/apt,710,30,3,2021-10-03,0.15,1,259,0,HSR21-001582,38340.0
88,8689844,Charming 1Bedroom House in Brentwood,9950113,Charleen,City of Los Angeles,Brentwood,34.06964,-118.46401,Entire home/apt,165,30,66,2021-07-26,0.74,2,127,0,0,43956.0
99,23286848,"Sweet studio in\nheart of UCLA,Brentwood,Westwood",27345195,Arman,City of Los Angeles,Westwood,34.06934,-118.46319,Entire home/apt,102,30,35,2020-08-13,0.57,2,89,0,HSR19-004956,20930.4
101,575334377654747671,"Worker’s paradise near UCLA, pool, balcony",417919018,Nicole,City of Los Angeles,Brentwood,34.06335,-118.46856,Entire home/apt,204,30,6,2022-12-21,0.51,2,32,6,0,37454.4


In [83]:
long_term_rentals["yearly_income"].mean()

36111.80844878049

In [84]:
# Filter by short term rental
short_term_rentals = listings_metadata.loc[listings_metadata["minimum_nights"]<30 ,:]
short_term_rentals.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license
6,51859630,"Single BR with King size bed, and shared bathroom",28407733,Arthur,City of Los Angeles,San Pedro,33.73928,-118.299110,Private room,55,2,3,2023-02-24,3.00,1,1,3,0
62,692376005462364487,Guesthouse in Los Ángeles,474760371,Mario,City of Los Angeles,Northridge,34.25615,-118.547287,Entire home/apt,175,2,13,2023-03-05,2.12,1,347,13,HSR22-002700
63,706447553284084730,Luxurious Villa w/ Lap Pool Outdoor Kitchen,410194448,Andrik,City of Los Angeles,Granada Hills,34.26513,-118.509430,Entire home/apt,725,2,10,2023-02-23,2.94,24,294,10,HSR22-003535
93,25167689,The Valley Guest House,128734546,Armando And Fanny,City of Los Angeles,Reseda,34.20728,-118.547200,Entire home/apt,168,1,149,2023-02-12,2.59,1,148,34,HSR20-002535
95,564398776260061956,Stylish 3-Bedroom Charmer - Convenient Location,445528454,Mary,City of Los Angeles,Encino,34.14815,-118.470560,Entire home/apt,350,3,3,2023-01-03,0.45,1,342,3,HSR22-000841


In [88]:
str_average_income = short_term_rentals["price"]*short_term_rentals["minimum_nights"]*short_term_rentals["reviews_per_month"]*12
short_term_rentals["yearly_income"] = str_average_income
short_term_rentals

/var/folders/mr/nzdp38cj3cz3b0gcgyvws4c40000gp/T/ipykernel_81479/2691676942.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  short_term_rentals["yearly_income"] = str_average_income


,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license,yearly_income
6,51859630,"Single BR with King size bed, and shared bathroom",28407733,Arthur,City of Los Angeles,San Pedro,33.739280,-118.299110,Private room,55,2,3,2023-02-24,3.00,1,1,3,0,3960.00
62,692376005462364487,Guesthouse in Los Ángeles,474760371,Mario,City of Los Angeles,Northridge,34.256150,-118.547287,Entire home/apt,175,2,13,2023-03-05,2.12,1,347,13,HSR22-002700,8904.00
63,706447553284084730,Luxurious Villa w/ Lap Pool Outdoor Kitchen,410194448,Andrik,City of Los Angeles,Granada Hills,34.265130,-118.509430,Entire home/apt,725,2,10,2023-02-23,2.94,24,294,10,HSR22-003535,51156.00
93,25167689,The Valley Guest House,128734546,Armando And Fanny,City of Los Angeles,Reseda,34.207280,-118.547200,Entire home/apt,168,1,149,2023-02-12,2.59,1,148,34,HSR20-002535,5221.44
95,564398776260061956,Stylish 3-Bedroom Charmer - Convenient Location,445528454,Mary,City of Los Angeles,Encino,34.148150,-118.470560,Entire home/apt,350,3,3,2023-01-03,0.45,1,342,3,HSR22-000841,5670.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42388,36835876,Contemporary Masterpiece Walls of Glass,4050393,Terry,City of Los Angeles,Sherman Oaks,34.171310,-118.442130,Entire home/apt,660,10,3,2021-05-24,0.09,3,251,0,HSR19-003705,7128.00
42396,792745924863986121,Huge 2 level LOFT with cabin Vibes in Downtown LA,21013529,The Rosemary Hospitality,City of Los Angeles,Downtown,34.045456,-118.252238,Entire home/apt,216,1,6,2023-02-25,4.39,3,99,6,HSR22-004071,11378.88
42402,640456703786612783,"Downtown LA APT, 2 bed 2 bath 20th floor!!",345776305,M,City of Los Angeles,Downtown,34.043190,-118.258450,Entire home/apt,386,2,16,2023-02-12,1.76,1,188,16,0,16304.64
42408,48933711,Mid-Century Modern Craftsman Hillside Home,55283847,Alyse,City of Los Angeles,Highland Park,34.127600,-118.191580,Entire home/apt,221,2,67,2023-03-07,5.30,1,173,64,HSR22-000166,28111.20


In [87]:
short_term_rentals["yearly_income"].mean()

17067.829974542932